<h1>3장 Pandas로 데이터 가공하기</h1>

<h2>데이터 세트 결합하기: 병합과 조인</h2>

<b>관계 대수</b>

pd.merge()에는 관계 데이터(relational data)를 조작하는 규칙의 정형 집합이자, 대부분의 데이터베이스에서 사용할 수 있는 연산의 개념적 기반을 형성하는 관계대수(relational algebra)의 하위 집합에 해당하는 행위가 구현돼 있다. 

<b>조인 작업의 분류</b>

pd.merge() 함수는 일대일, 다대일, 다대다 조인같은 여러 조인을 구현할 수 있다.

<b>일대일 조인</b>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.DataFrame({'employee': ['Bob','Jake','Lisa','Sue'],
               'group': ['Accounting','Engineering','Engineering','HR']})
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [3]:
df2 = pd.DataFrame({'employee': ['Lisa','Bob','Jake','Sue'],
               'hire_date': ['2004','2008','2012','2014']})
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [4]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


<b>다대일(Many-to-one) 조인</b>

In [5]:
df4 = pd.DataFrame({'group': ['Accounting','Engineering','HR'],
               'supervisor': ['Carly','Guido','Steve']})
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [6]:
pd.merge(df3, df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


<b>다대다(Many-to-many)조인</b>

In [7]:
df5 = pd.DataFrame({'group': ['Accounting','Accounting','Engineering','Engineering','HR','HR'],
             'skills': ['marh','speadsheets','coding','linux','spreadsheeds','organization']})
df5

,group,skills
0,Accounting,marh
1,Accounting,speadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheeds
5,HR,organization


In [8]:
pd.merge(df1, df5)

,employee,group,skills
0,Bob,Accounting,marh
1,Bob,Accounting,speadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheeds
7,Sue,HR,organization


<b>병합 키 지정</b>

<b>on 키워드</b>

In [19]:
print(df1);print(df2);
pd.merge(df1, df2, on='employee')

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee hire_date
0     Lisa      2004
1      Bob      2008
2     Jake      2012
3      Sue      2014


,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


<b>left_on과 right_on 키워드</b>

In [10]:
df3 = pd.DataFrame({'name': ['Bod','Jake','Lisa','Sue'],
                   'salary':[70000, 80000, 120000, 90000]})
df3

,name,salary
0,Bod,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [20]:
pd.merge(df1, df3, left_on = "employee", right_on="name")

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,900000


In [13]:
pd.merge(df1, df3, left_on = "employee", right_on="name").drop('name', axis = 1)

,employee,group,salary
0,Jake,Engineering,80000
1,Lisa,Engineering,120000
2,Sue,HR,90000


<b>left_index와 right_index 키워드</b>

In [22]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a); print(df2a); 

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
         hire_date
employee          
Lisa          2004
Bob           2008
Jake          2012
Sue           2014


In [24]:
pd.merge(df1a, df2a, left_index = True, right_index= True)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


기본적인 인덱스 기반으로 조인하는 병합을 수행하는 join()을 사용하기도 한다.

In [25]:
print(df1a.join(df2a))

                group hire_date
employee                       
Bob        Accounting      2008
Jake      Engineering      2012
Lisa      Engineering      2004
Sue                HR      2014


In [29]:
pd.merge(df1a, df3, left_index = True, right_on = 'name')

,group,name,salary
0,Accounting,Bob,70000
1,Engineering,Jake,80000
2,Engineering,Lisa,120000
3,HR,Sue,900000


<b>조인을 위한 집합 연산 지정하기</b>

In [46]:
df6 =  pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns = ['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                    columns = ['name', 'drink'])
print(df6); print(df7)

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer


In [47]:
print(pd.merge(df6, df7))

   name   food drink
0  Mary  bread  wine


name에서 공통한목으로 'Mary'를 가지고 있는 두 데이터 세트를 병합 : 기본적으로 교집합, 내부조인(inner join)

In [48]:
 pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


In [49]:
 pd.merge(df6, df7, how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


In [50]:
 pd.merge(df6, df7, how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


In [51]:
 pd.merge(df6, df7, how='right')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


<b>열 이름이 겹치는 경우: sufflexs 키워드</b>

In [52]:
df8 =  pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1,2,3,4] })
df9 =  pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3,1,4,2] })
print(df8); print(df9)

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2


In [53]:
print(pd.merge(df8, df9, on = "name"))

   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


In [54]:
print(pd.merge(df8, df9, on = "name", suffixes=["_L", "_R"]))

   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


<b>예제: 미국 주 데이터</b>

In [70]:
#!curl -0 https://raw.githubusercontent.com/jakevdp/PythonDataScienceHandbook/master/notebooks/data/state-population.csv

In [71]:
pop = pd.read_csv('/Users/chosikc/Sites/PythonDataHandbook/state-population.csv')
areas = pd.read_csv('/Users/chosikc/Sites/PythonDataHandbook/state-areas.csv')
abbrevs = pd.read_csv('/Users/chosikc/Sites/PythonDataHandbook/state-abbrevs.csv')

In [72]:
print(pop.head());print(areas.head());print(abbrevs.head());

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [77]:
merged = pd.merge(pop, abbrevs, how = 'outer', left_on = 'state/region', right_on = 'abbreviation')
merged = merged.drop('abbreviation', 1) # 중복 정보 삭제
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [78]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [79]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [80]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [81]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [83]:
final = pd.merge(merged, areas, on = 'state', how = 'left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [84]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [86]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [87]:
final.dropna(inplace=True) #NULL 값 삭제
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [89]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [91]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']
density.sort_values(ascending =False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [93]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64